In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [35]:
from IPython.display import HTML
style = "<style>h2{color: #FF756D; font-size: 400%} h3 {color: #85DE77;font-size: 200%;} h4 {color: #FFF49C; font-size: 150%} p {color:#F9FFCB }</style>"
print('Just stylising the markdowns')
HTML(style)

Just stylising the markdowns


## Reading in train and test data and finding out what we have here
### Colour scheme for markdown:
Coral Reef (#FF756D), Pastel Yellow (#FFF49C), Conditioner (#F9FFCB) and Pastel Green (#85DE77)


In [4]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df.shape

(1460, 81)

#### There are 81 features! How should I select and process them for the model?

## Preprocessing
### Finding out missing data and data types

In [33]:
missingCounts = df.isnull().sum()
missingNormalised = df.isnull().sum()/df.isnull().count()*100
print('There are {} columns with missing data'.format((missingCounts!=0).sum()))
missingNormalised.sort_values(ascending = False)[missingCounts!=0]

There are 19 columns with missing data


PoolQC          99.520548
MiscFeature     96.301370
Alley           93.767123
Fence           80.753425
FireplaceQu     47.260274
LotFrontage     17.739726
GarageCond       5.547945
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageQual       5.547945
BsmtExposure     2.602740
BsmtFinType2     2.602740
BsmtFinType1     2.534247
BsmtCond         2.534247
BsmtQual         2.534247
MasVnrArea       0.547945
MasVnrType       0.547945
Electrical       0.068493
dtype: float64

In [48]:
continuous = df.columns[df.dtypes!= object]
categorical = df.columns[df.dtypes == object]
featureTypes = pd.DataFrame([continuous,categorical], index = ['Continuous','Categorical'])
featureTypes.T

,Continuous,Categorical
0,Id,MSZoning
1,MSSubClass,Street
2,LotFrontage,Alley
3,LotArea,LotShape
4,OverallQual,LandContour
5,OverallCond,Utilities
6,YearBuilt,LotConfig
7,YearRemodAdd,LandSlope
8,MasVnrArea,Neighborhood
9,BsmtFinSF1,Condition1


In [54]:
df.var()

Id               1.777550e+05
MSSubClass       1.789338e+03
LotFrontage      5.897492e+02
LotArea          9.962565e+07
OverallQual      1.912679e+00
OverallCond      1.238322e+00
YearBuilt        9.122154e+02
YearRemodAdd     4.262328e+02
MasVnrArea       3.278497e+04
BsmtFinSF1       2.080255e+05
BsmtFinSF2       2.602391e+04
BsmtUnfSF        1.952464e+05
TotalBsmtSF      1.924624e+05
1stFlrSF         1.494501e+05
2ndFlrSF         1.905571e+05
LowQualFinSF     2.364204e+03
GrLivArea        2.761296e+05
BsmtFullBath     2.692682e-01
BsmtHalfBath     5.700283e-02
FullBath         3.035082e-01
HalfBath         2.528937e-01
BedroomAbvGr     6.654938e-01
KitchenAbvGr     4.854892e-02
TotRmsAbvGrd     2.641903e+00
Fireplaces       4.155947e-01
GarageYrBlt      6.095825e+02
GarageCars       5.584797e-01
GarageArea       4.571251e+04
WoodDeckSF       1.570981e+04
OpenPorchSF      4.389861e+03
EnclosedPorch    3.735550e+03
3SsnPorch        8.595059e+02
ScreenPorch      3.108889e+03
PoolArea  

In [55]:
df.PoolArea.value_counts()

0      1453
738       1
648       1
576       1
555       1
519       1
512       1
480       1
Name: PoolArea, dtype: int64